In [1]:
from sklearn import datasets 
import matplotlib.pyplot as plt 
import seaborn as sns
import math
import numpy as np
from scipy import stats
from tqdm import tqdm

data = datasets.load_iris()
data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [3]:
X,Y = data['data'],data['target']

In [4]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesRegressor
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=10)
model.fit(X,Y)
print(accuracy_score(Y,model.predict(X)))


0.9933333333333333


In [133]:
ExtraTreesRegressor?

# My Random Forest Implementation 

In [5]:
def get_random_subsets(X, y, n_subsets, replacements=True):
    
    n_samples = np.shape(X)[0]
    # Concatenate x and y and do a random shuffle
    X_y = np.concatenate((X, np.expand_dims(y,axis = 1)), axis=1)
    np.random.shuffle(X_y)
    subsets = []

    # Uses 50% of training samples without replacements
    subsample_size = int(n_samples // 2)
    if replacements:
        subsample_size = n_samples      # 100% with replacements

    for _ in range(n_subsets):
        idx = np.random.choice(
            range(n_samples),
            size=subsample_size,
            replace=replacements)
        
        X = X_y[idx][:, :-1]
        y = X_y[idx][:, -1]
        
        subsets.append([X, y])
    return subsets

In [20]:
class RandomForest:
    
    def __init__(self,n_estimators = 3,min_samples_split=3, min_impurity=1e-7,
                 max_depth= 5,criterian = 'entropy',max_features = 'sqrt'):
        
        self.n_estimators = n_estimators
        self.tree = []
        self.feature_idx = []
        self.max_features = max_features
        
        for i in range(self.n_estimators):
        
            self.tree.append(DecisionTreeClassifier(min_samples_split,
                                                     min_impurity,
                                                     max_depth,criterian,
                                                    ))
        
        
    def fit(self,X,Y):    
        
        n_samples,n_features = X.shape
        
        
        #     - If "auto", then `max_features=sqrt(n_features)`.
        #     - If "sqrt", then `max_features=sqrt(n_features)` (same as "auto").
        #     - If "log2", then `max_features=log2(n_features)`.
        
        #number of features to consider 
        
        if self.max_features == None:
            self.max_features = n_features
        elif self.max_features == 'sqrt':
            self.max_features = np.sqrt(n_features).astype('int32')
        else:
            self.max_features = np.log2(n_features).astype('int32')
            
    
        
        #random forest uses bagging in which we 
        #1. need to create sub-samples of the dataset : bootstraping 
        #2. then train each tree on a unique tree and aggregate the results 
        
        subsets  = get_random_subsets(X,Y,self.n_estimators)
            
#         print(self.max_features,type(self.max_features))    
        
        for i in range(self.n_estimators):
            
            #get the sampled dataset 
            X_,Y_  = subsets[i]
            
            #choose random features 
            
            idx = np.random.choice(range(n_features), size=self.max_features, replace=False)
            # Save the indices of the features for prediction
            
            self.feature_idx.append(idx)
            
            # Choose the features corresponding to the indices
            X_ = X_[:, idx]
          
            self.tree[i].fit(X_,Y_)
        
        
    def predict(self,X):
        
        preds = np.zeros((X.shape[0],self.n_estimators))
        for i in range(self.n_estimators):
            
            #select the proper indices 
            X_ = X[:,self.feature_idx[i]]
            
            preds[:,i] = self.tree[i].predict(X_)
        
        final_pred = []
        for pred in preds:
            
            final_pred.append(majority_vote(pred))
        
        
#         return np.round(np.mean(preds,1))

        return final_pred
        

In [24]:
model = RandomForest(n_estimators= 10,criterian='gini')
model.fit(X,Y)
# model.predict(X)
accuracy_score(Y,model.predict(X))

0.9866666666666667

In [9]:
 np.random.choice(range(4), size=2, replace=True)

array([2, 2])

# Base Model

In [17]:
class DecisionNode:
    
    def __init__(self,feature = None,
                     entropy = None,threshold = None,
                     value = None,right=None,left=None):   
            
            self.entropy = entropy
            
            self.feature = feature
             
            self.right = right
            
            self.left = left
            
            self.threshold = threshold
            
            self.value = value

# function to find feature to split on 

def divide_on_feature(X, feature_i, threshold):

    split_func = None
    if isinstance(threshold, int) or isinstance(threshold, float):
        split_func = lambda sample: sample[feature_i] >= threshold
    else:
        split_func = lambda sample: sample[feature_i] == threshold

    X_1 = np.array([sample for sample in X if split_func(sample)])
    X_2 = np.array([sample for sample in X if not split_func(sample)])

    return np.array([X_1, X_2])


def entropy(y):
    
    classes = np.unique(y) # 1 2 3
    length = len(y)
    entropy = 0
    for label in classes:
        p = np.sum(y == label) / length
        entropy += p * np.log2(p)
        
    return -entropy


def gini_index(y):
    
    # formula : 1 - p+ ^ 2 - p-^2 
    
    classes = np.unique(y) # 1 2 3
    length = len(y)
    gini = 1
    proba = 0
    for label in classes:
        p = np.sum(y == label) / length
        proba += p **2
        
    return gini - proba

    



    
def majority_vote(y):
    
        most_common = None
        max_count = 0
        for label in np.unique(y):
            # Count number of occurences of samples with label
            count = len(y[y == label])
            if count > max_count:
                most_common = label
                max_count = count
        return most_common

    

class DecisionTreeClassifier:
    
    
    def __init__(self, min_samples_split=3, min_impurity=1e-7,
                 max_depth= 5,criterian = 'entropy'):
        
        #contains the root of the tree 
        self.root = None         
        self.criterian = criterian
        #stopping conditions 
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_impurity = min_impurity
        
        
        
        
    def fit(self,X,y):
        
        self.one_dim = len(np.shape(y)) == 1

        
#         print(f'Shape of dataset :: {X.shape}')
        
        self.root  = self._build_tree(X,y)
        
    
        
    def _build_tree(self, X, y, current_depth=0):
       

        largest_impurity = 0
        best_criteria = None    # Feature index and threshold
        best_sets = None     # Subsets of the data
        right = None 
        left= None
        selected_feature = None
        selected_threshold = None
        
        # Check if expansion of y is needed
        if len(np.shape(y)) == 1:
            y = np.expand_dims(y, axis=1)

        # Add y as last column of X
        Xy = np.concatenate((X, y), axis=1)

        n_samples, n_features = np.shape(X)

        
        if n_samples >= self.min_samples_split and self.max_depth >= current_depth:
            
#             print(current_depth)
            
            # Calculate the impurity for each feature
            for feature_i in range(n_features):
                
                # All values of feature_i
                feature_values = np.expand_dims(X[:, feature_i], axis=1)
                unique_values = np.unique(feature_values)

                # Iterate through all unique values of feature column i and
                # calculate the impurity
                for threshold in unique_values:
                    # Divide X and y depending on if the feature value of X at index feature_i
                    # meets the threshold
                    Xy1, Xy2 = divide_on_feature(Xy, feature_i, threshold)

                    if len(Xy1) > 0 and len(Xy2) > 0:
                        # Select the y-values of the two sets
                        y1 = Xy1[:,-1]
                        y2 = Xy2[:,-1]

                        # Calculate impurity
                        impurity = self.info_gain(y, y1, y2)

                        # If this threshold resulted in a higher information gain than previously
                        # recorded save the threshold value and the feature
                        # index
                        if impurity > largest_impurity:
                            largest_impurity = impurity
                            selected_feature = feature_i
                            selected_threshold = threshold 
                            right,left = Xy2,Xy1
    
    
        if largest_impurity > self.min_impurity:
            # Build subtrees for the right and left branches
            left = self._build_tree(left[:,:-1], left[:,-1], current_depth = current_depth + 1)
            right = self._build_tree(right[:,:-1], right[:,-1], current_depth = current_depth + 1)
            return DecisionNode(feature=selected_feature, threshold=selected_threshold, left=left, right=right)

        # We're at leaf => determine value
        leaf_value = majority_vote(y)

        return DecisionNode(value=leaf_value)
    
    
    
    def get_imputiry_metric(self):
        
        if self.criterian == 'entropy':
            return entropy
        else:
            return gini_index
            
    
    def info_gain(self,y,y1,y2):
    
        p = len(y1) / len(y)
              
        criterian = self.get_imputiry_metric()

        ent = criterian(y)

        info_gain = ent - p * criterian(y1) - (1 - p) * criterian(y2)

        return info_gain
    
    
    
    

    def predict_value(self, x, tree=None):
        

        if tree is None:
            tree = self.root

        if tree.value is not None:
            return tree.value

        feature_value = x[tree.feature]

        # Determine if we will follow left or right branch
        branch = tree.right
        if isinstance(feature_value, int) or isinstance(feature_value, float):
            if feature_value >= tree.threshold:
                branch = tree.left
        elif feature_value == tree.threshold:
            branch = tree.left


        # Test subtree
        return self.predict_value(x, branch)

        
    def predict(self,X):
        
        y_pred = [self.predict_value(sample) for sample in X]
         
        return y_pred
        
    

RandomForest Vs ExtraTreeClfs

![](Q18mk.png)

ExtraTreesClassifier is like a brother of RandomForest but with 2 important differences.

We are building multiple decision trees. For building multiple trees, we need multiple datasets. Best practice is that we don't train the decision trees on the complete dataset but we train only on fraction of data (around 80%) for each tree. In a random forest, we draw observations with replacement. So we can have repetition of observations in a random forest. In an ExtraTreesClassifier, we are drawing observations without replacement, so we will not have repetition of observations like in random forest.

The split is the process of converting a non-homogeneous parent node into 2 homogeneous child node (best possible). In RandomForest, it select the best split to convert the parent into the two most homogeneous child nodes. In an ExtraTreesClassifier, it selects a random split to divide the parent node into two random child nodes.

Let’s look at some ensemble methods ordered from high to low variance, ending in ExtraTreesClassifier.



ExtraTreesClassifier is like a brother of RandomForest but with 2 important differences.

enter image description here

We are building multiple decision trees. For building multiple trees, we need multiple datasets. Best practice is that we don't train the decision trees on the complete dataset but we train only on fraction of data (around 80%) for each tree. In a random forest, we draw observations with replacement. So we can have repetition of observations in a random forest. In an ExtraTreesClassifier, we are drawing observations without replacement, so we will not have repetition of observations like in random forest.

The split is the process of converting a non-homogeneous parent node into 2 homogeneous child node (best possible). In RandomForest, it select the best split to convert the parent into the two most homogeneous child nodes. In an ExtraTreesClassifier, it selects a random split to divide the parent node into two random child nodes.

Let’s look at some ensemble methods ordered from high to low variance, ending in ExtraTreesClassifier.

1. Decision Tree (High Variance)

A single decision tree is usually overfits the data it is learning from because it learn from only one pathway of decisions. Predictions from a single decision tree usually don’t make accurate predictions on new data.

2. Random Forest (Medium Variance)

Random forest models reduce the risk of overfitting by introducing randomness by:

    building multiple trees (n_estimators)
    drawing observations with replacement (i.e., a bootstrapped sample)
    splitting nodes on the best split among a random subset of the features selected at every node. Split is process to convert non-homogeneous parent node into 2 homogeneous child node(best possible).

3. Extra Trees (Low Variance)

Extra Trees is like a Random Forest, in that it builds multiple trees and splits nodes using random subsets of features, but with two key differences: it does not bootstrap observations (meaning it samples without replacement), and nodes are split on random splits, not best splits. So in summary, ExtraTrees:

    builds multiple trees with bootstrap = False by default, which means it samples without replacement
    nodes are split based on random splits among a random subset of the features selected at every node


# to make RandomForest Act like decision trees 
1. set replacement = False
2. split on a random sampple from a fraction of features 